In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
#importando datasets http://www.respeitoavida.sp.gov.br/relatorios/
df_obitos_raw=pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\obitos_publico.csv',encoding='unicode_escape',sep=';')
df_nao_fatais_raw = pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_naofatais.csv',encoding='unicode_escape',sep=';')
df_fatais_raw = pd.read_excel(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_fatais.xlsx')

C:\Users\Renato\anaconda3\envs\Data_Science\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#checando duplicatas
df_obitos_raw.duplicated().value_counts() #achado 97 linhas duplicadas (os outros df's não tem duplicatas)
df_obitos_raw = df_obitos_raw.drop_duplicates()#dropando as 97 duplicatas, temos 34657 linhas

In [24]:
# Criando Dataframes para trabalhar
df_nao_fatais = df_nao_fatais_raw.copy()

df_fatais = df_fatais_raw.copy()#NAO TEMOS COMO FILTRAR POR MOTOCILCETA POIS NAO TEM ESSA COLUNA!!!!

df_obitos = df_obitos_raw.copy()#AQUI CONSEGUIMOS FILTRAR POR MOTOCICLETA

In [25]:
#checando sempre a quantidade de dados dos dataframes
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 55)


In [26]:
#Arrumando o nome das colunas
df_obitos = df_obitos_raw.rename(columns={'LAT_(GEO)':'Latitude',
                                          'LONG_(GEO)':'Longitude',
                                          'Tipo de via':'Tipo de Via',
                                         'Id da Delegacia (RDO)':'Id Delegacia', 'Número do Bo (RDO)':'Número BO',
                                          'Ano do BO (RDO)':'Ano BO','Númeral / KM':'Numero/KM'})

df_nao_fatais = df_nao_fatais_raw.rename(columns={'LAT_(GEO)':'Latitude','LONG_(GEO)':'Longitude'})


df_fatais =df_fatais_raw.rename(columns={'Lat (GEO)':'Latitude','Long (GEO)':'Longitude','Numeral / KM':'Numero/KM',
                                         'Iluminação da via (SIOPM)':'Iluminação',
                                        'Superfície da Via (SIOPM)':'Superfície da via', 
                                         'Dia da semana':'Dia da Semana',
                                         'Tipo de via':'Tipo de Via',
                                         'Tipo de pista (SIOPM)':'Tipo de pista',
                                         'Id Delegacia (RDO)':'Id Delegacia', 'Número BO (RDO)':'Número BO', 
                                         'Ano BO (RDO)':'Ano BO',
                                        'Condições Climáticas (SIOPM)':'Condições Climáticas'})

In [27]:
# dropando colunas duplicada
# no df_nao_fatais, colunas 'Condições da pista' e 'Tipo de pista', portando dropamos a coluna 'Condições da pista'
df_nao_fatais.drop(['Condições da pista'],axis=1,inplace=True)

In [28]:
#alterando tipos de variáveis
df_obitos['Data do Acidente'] = pd.to_datetime(df_obitos['Data do Acidente'],errors='coerce')

df_nao_fatais['Data do Acidente'] = pd.to_datetime(df_nao_fatais['Data do Acidente'],format='%Y-%m-%d')
df_nao_fatais['Ano do Acidente'] = pd.to_datetime(df_nao_fatais['Ano do Acidente'],format='%Y')
df_nao_fatais['Dia do Acidente'] = pd.to_datetime(df_nao_fatais['Dia do Acidente'],format='%d')

df_fatais.loc[31790,'Data do Acidente']='2020-02-27'
df_fatais['Data do Acidente'] = pd.to_datetime(df_fatais['Data do Acidente'],format='%Y-%m-%d')
df_fatais.loc[31790, 'Ano do Acidente']='2020'
df_fatais['Ano do Acidente'] = pd.to_datetime(df_fatais['Ano do Acidente'],format='%Y')
df_fatais['Dia do Acidente'] = pd.to_datetime(df_fatais['Dia do Acidente'],format='%d')

In [32]:
#juntando a df_fatais_geral com df_nao_fatais, criando df_acidentes
df_fatais_geral = df_fatais.merge(df_obitos,
                                  on=['Número BO','Data do Acidente','Ano BO','Hora do Acidente','Turno','Município','Região Administrativa','Jurisdição','Administração','Conservação','Outro Veículo Envolvido','Tipo de Via','Logradouro','Dia da Semana','Numero/KM','Id Delegacia','Latitude','Longitude'])

In [33]:
#checando sempre a quantidade de dados dos dataframes
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)
print('merge',df_fatais_geral.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 54)
merge (33934, 48)


In [31]:
df_fatais_geral.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33934 entries, 0 to 33933
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   ID                                   33934 non-null  int64         
 1   Id Delegacia                         33934 non-null  int64         
 2   Número BO                            33934 non-null  int64         
 3   Ano BO                               33934 non-null  int64         
 4   Data do Acidente                     33934 non-null  datetime64[ns]
 5   Dia do Acidente                      33934 non-null  datetime64[ns]
 6   Mês do Acidente                      33934 non-null  object        
 7   Ano do Acidente                      33934 non-null  datetime64[ns]
 8   Ano/Mês do Acidente                  33934 non-null  float64       
 9   Dia da Semana                        33934 non-null  object        
 10  Hora do Ac

In [39]:
#checando o pq algumas variáveis acima tem valores nulos. 
df_fatais_geral['Outro Veículo Envolvido'].value_counts()
# Temos apenas 22544 valores não nulos de 33934. Podemos considerar essa diferença de nulos como da categoria "Não Há"?? E os "não disponíveis" podem ser considerados "Não Há"?? Se sim fazer um apply lambda pra corrigir isso. CHECAR COM O GRUPO!!!

AUTOMOVEL         7650
NÃO HÁ            4561
NAO DISPONIVEL    3503
CAMINHAO          3450
MOTOCICLETA       1724
ONIBUS            1376
OUTROS             143
BICICLETA           94
PEDESTRE            43
Name: Outro Veículo Envolvido, dtype: int64

In [40]:
df_fatais_geral['Sub Tipo do Acidente'].value_counts()
#aqui quase 50% dos dados são nulos. E temos muitas categorias....CHECAR COM O GRUPO!!!

NAO DISPONIVEL                           3512
FRONTAL                                  2136
CRUZANDO A VIA                           1858
TRASEIRA                                 1382
LATERAL                                  1144
TOMBAMENTO VEÍCULO DE 2 RODAS            1142
POSTE                                     813
TRANSVERSAL                               782
CAPOTAMENTO                               695
ANDANDO NA VIA / ACOSTAMENTO              523
ÁRVORE                                    464
DEFENSA / BARREIRA                        440
MURO/PAREDE/TALUDE                        406
VEÍCULO ESTACIONADO                       399
OUTROS                                    265
CONDUTOR/PASSAGEIROS FORA DO VEÍCULO      218
QUEDA (ÁGUA, BARRANCO, PONTE)             176
NA CALÇADA                                163
TOMBAMENTO VEÍCULO DE MAIS DE 2 RODAS     154
MEIO-FIO/CALÇAMENTO/DRENAGEM              150
ANIMAL                                    109
ENGAVETAMENTO                     

In [41]:
#criando colunas com labels de fatal e não falal antes de juntar
df_fatais_geral['Acidente Fatal']=1
df_fatais_geral['Acidente Não Fatal']=0
df_nao_fatais['Acidente Fatal']=0
df_nao_fatais['Acidente Não Fatal']=1


In [42]:
#juntando a df_fatais_geral com df_nao_fatais criando df_acidentes
df_acidentes = pd.concat([df_nao_fatais,df_fatais_geral])

In [43]:
df_acidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411547 entries, 0 to 33933
Data columns (total 80 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   ID                                           411547 non-null  int64         
 1   Data do Acidente                             411547 non-null  datetime64[ns]
 2   Dia do Acidente                              411547 non-null  datetime64[ns]
 3   Mês do Acidente                              411547 non-null  object        
 4   Ano do Acidente                              411547 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          411547 non-null  float64       
 6   Dia da Semana                                411547 non-null  object        
 7   Hora do Acidente                             411547 non-null  object        
 8   Turno                                        411547 non-null  obj

In [47]:
#checando sempre a quantidade de dados dos dataframes
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)

df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 56)
df_fatais_geral (33934, 50)
df_acidentes (411547, 80)


In [48]:
df_acidentes['Mão de direção'].value_counts()

NAO DISPONIVEL    167791
DUPLA             108445
ÚNICA             101375
NÃO                    2
Name: Mão de direção, dtype: int64

In [51]:
df_acidentes['Local do óbito'].value_counts()

ESTABELECIMENTO DE SAUDE    18213
VIA                         13865
OUTROS                       1341
NAO DISPONIVEL                515
Name: Local do óbito, dtype: int64

In [60]:
#FILTROS
# Filtrando por Região metropolitana de SP
df_acidentes_sp = df_acidentes.loc[df_acidentes['Região Administrativa']=='Metropolitana de São Paulo']#
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']>='2019-01-01']
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']<='2020-12-31']

In [71]:
#filtrando por acidentes que envolvem motocicleta
# criando dataframes preliminares com diferentes filtros
df_acidentes_sp_moto1 = df_acidentes_sp.loc[df_acidentes_sp['Veículos Envolvidos - Motocicleta']==1] #dados que vem do "não fatais" onde temos motos envolvidas
df_acidentes_sp_moto2 = df_acidentes_sp.loc[df_acidentes_sp['Tipo do veículo da vítima']=='MOTOCICLETA'] # dados que vem dos "fatais", que são os óbitos de motocicleta
df_acidentes_sp_moto3 = df_acidentes_sp.loc[df_acidentes_sp['Outro Veículo Envolvido']=='MOTOCICLETA'] # dados que vem dos "fatais" , que podem ter motos como outo veículo envolvido
#concatenando os 3 dataframes
df_acidentes_sp_moto = pd.concat([df_acidentes_sp_moto1,df_acidentes_sp_moto2,df_acidentes_sp_moto3])
#checando duplicatas
df_acidentes_sp_moto.duplicated().value_counts() #169 duplicatas pois tem como motocicletas o 'Tipo do veículo da vítima' e 'Outro Veículo Envolvido', do df_fatais. deletar!
df_acidentes_sp_moto = df_acidentes_sp_moto.drop_duplicates()

In [73]:
df_acidentes_sp_moto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62479 entries, 0 to 33910
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   ID                                           62479 non-null  int64         
 1   Data do Acidente                             62479 non-null  datetime64[ns]
 2   Dia do Acidente                              62479 non-null  datetime64[ns]
 3   Mês do Acidente                              62479 non-null  object        
 4   Ano do Acidente                              62479 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          62479 non-null  float64       
 6   Dia da Semana                                62479 non-null  object        
 7   Hora do Acidente                             62479 non-null  object        
 8   Turno                                        62479 non-null  object        


In [74]:
#checando sempre a quantidade de dados dos dataframes
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)
print('df_acidentes_sp',df_acidentes_sp.shape)
print('df_acidentes_sp_moto',df_acidentes_sp_moto.shape)


df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 56)
df_fatais_geral (33934, 50)
df_acidentes (411547, 80)
df_acidentes_sp (159898, 80)
df_acidentes_sp_moto (62479, 80)


In [77]:
# checando o balanceamento do dataset
positive_proportion = 100*df_acidentes_sp_moto['Acidente Fatal'].value_counts()[1]/len(df_acidentes_sp_moto)
print('Proporção de fatais:', round(positive_proportion,2),'%')

Proporção de fatais: 6.91 %


In [ ]:
#PRECISAMOS CHECAR COLUNAS ANTES DO REPLACE NAN

In [ ]:
#Substituir os nao disponiveis por nulos
# df_acidentes_sp_moto.replace('NAO DISPONIVEL', np.nan,inplace=True)

In [ ]:
# Selecionar colunas pra exportar


In [ ]:
# export
df_acidentes_moto_sp.to_csv('df_acidentes_moto_sp.csv')